In [1]:
# Basic libraries
import pandas as pd
import numpy as np
import math

# Models
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Cross-Validation
from sklearn.model_selection import GroupKFold

# Prediction Scoring
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import make_scorer
from statistics import mean

# Ignore warnings
import warnings
warnings.simplefilter('ignore')  

### Loading the test group for prediction

In [2]:
# Load Features created in StartFromScratch.ipynb
df = pd.read_csv('Final_Features_test.csv')

# Scale data for better perfomance
scaler = StandardScaler()

# Extract name of columns in df as a list
columns = list(df.columns)

# Eliminate both participant name and empathy column because they are not a features
columns.pop(0)  
columns.pop(-1)

# Scale data by columns 
df[columns] = scaler.fit_transform(df[columns])

### Categorizing our label
We are passing our label from a continuous to a discrete variable for a better __interpretation__ of the results and for the __simplicity__ of the predictions. We use the `pd.cut` function from pandas to create the bins and the respective label. In __Table 1__, we explain the process done.

In [3]:
# Discretizing my label/target
df['Empathy Score'] = pd.cut(df["Empathy Score"],
       bins=[80, 100, 110, 120, 130], 
       labels=[0, 1, 2, 3])

### <center> Table 1
| Empathy Range | Class | Label |
| :-: | :-: | :-: |
| <100 | Bad | 0 |
| 100-110 | Average | 1 |
| 110-120 | Good | 2 |
| >120 | Outstanding | 3 |

In [4]:
# Print DataFrame to see the results
df.head()

,Participant name,Mean Pupil diameter left,Std Pupil diameter left,Mean Pupil diameter right,Std Pupil diameter right,Num. of Fixations,Num. of Saccades,Num. of Unclassified,Recording duration (s),Mean Gaze event duration (s),Mean Fixation point X,Std Fixation point X,Mean Fixation point Y,Std Fixation point Y,Mean Gaze point X,Std Gaze point X,Mean Gaze point Y,Std Gaze point Y,Empathy Score
0,1.0,-0.089633,-0.814297,-0.196801,-0.876958,-0.098426,0.379303,-0.355026,-0.204618,-0.293486,1.662709,0.222486,-1.385322,-0.507767,1.866861,0.241699,-1.953166,-0.962940,3
1,1.0,-0.204996,-0.575084,-0.247813,-0.750402,-0.123609,0.038046,-0.614439,-0.413981,-0.315063,0.822417,1.022072,-1.011522,-0.429986,0.872312,1.171768,-1.211816,-1.184586,3
2,1.0,-0.162118,-1.071167,-0.142893,-1.213951,-0.474775,-0.019179,-0.614439,-0.615296,-0.421633,0.638070,0.134267,-0.326136,-0.123241,0.638859,0.031482,-0.480517,-0.398963,3
3,1.0,-0.251455,-0.969113,-0.234711,-1.115633,0.374458,0.709393,-0.370985,0.221591,-0.388220,0.550088,-0.864774,-0.419745,-0.033322,0.650658,-1.107551,-0.456417,-0.585927,3
4,1.0,-0.277955,-0.826529,-0.230868,-1.018470,-0.255401,0.046421,-0.564525,-0.453132,-0.308629,0.921692,0.202833,-0.267798,-0.243033,1.071226,0.286408,-0.341328,-0.583934,3


### Split data using KFolds
We are splitting our training and testing data using Group KFolds because of the nature of our labels, in essence we have a bunch of rows that that share the same label so we dont want any leakage of data from the testing data to the training data. In this code we separate the features and our label first and then create the k-folds.

In [5]:
# Drop participant name and empathy column because they are not a feature
X = df.drop(['Participant name','Empathy Score'], axis=1)

# Store label in variable 'y'
y = df['Empathy Score']

# Store group indexes in 'gps'
gps = df['Participant name']

# Create Group KFolds
kf = GroupKFold(n_splits=5)

In [6]:
for train_index, test_index in kf.split(X, y, groups=gps):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 17)
(73, 17)
(288,)
(73,)


## Classification Models

In [8]:
# DUMMY CLASSIFIER
dummy = DummyClassifier(strategy ='prior') 
dummy.fit(X_train, y_train)
y_pred_dummy = cross_val_score(dummy, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# DECISION TREE CLASSIFIER
# dt = DecisionTreeClassifier(criterion='gini',max_depth=5,min_samples_leaf=4,min_samples_split=7,random_state=13)
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = cross_val_score(dt, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# ADABOOST CLASSIFIER
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(X_train, y_train)
y_pred_ada = cross_val_score(ada, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# NN CLASSIFIER
nn = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(5, 2),random_state=1)
nn.fit(X_train, y_train)
y_pred_nn = cross_val_score(nn, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# SVM CLASSIFIER
svc = SVC(random_state=13)
svc.fit(X_train, y_train)
y_pred_svc = cross_val_score(svc, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# SGD CLASSIFIER
sgd = SGDClassifier(random_state=13)
sgd.fit(X_train, y_train)
y_pred_sgd = cross_val_score(sgd, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# KNN CLASSIFIER
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = cross_val_score(knn, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# NEAREST CENTROID
ncc = NearestCentroid()
ncc.fit(X_train, y_train)
y_pred_ncc = cross_val_score(ncc, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# RANDOM FORREST
rfr = RandomForestClassifier(n_estimators=300, random_state=13)
rfr.fit(X_train, y_train)
y_pred_rfr = cross_val_score(rfr, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# LOGISTIC REGRESSION
logreg = LogisticRegression(random_state=13)
logreg.fit(X_train, y_train)
y_pred_logreg = cross_val_score(logreg, X, y, cv=kf, groups=gps, scoring='balanced_accuracy')

# RESULTS
print('Accuracy SGDC:     %.2f +/- %.2f' % ((y_pred_sgd.mean()),y_pred_sgd.std()),'***','\n')
print('Accuracy NCC:      %.2f +/- %.2f' % ((y_pred_ncc.mean()),y_pred_ncc.std()))
print('Accuracy NN:       %.2f +/- %.2f' % ((y_pred_nn.mean()),y_pred_nn.std()))
print('Accuracy LOGR:     %.2f +/- %.2f' % ((y_pred_logreg.mean()),y_pred_logreg.std()),'\n')
print('Accuracy Adaboost: %.2f +/- %.2f' % ((y_pred_ada.mean()),y_pred_ada.std()))
print('Accuracy DT:       %.2f +/- %.2f' % ((y_pred_dt.mean()),y_pred_dt.std()))
print('Accuracy RFR:      %.2f +/- %.2f' % ((y_pred_rfr.mean()),y_pred_rfr.std()),'\n')
print('Accuracy KNN:      %.2f +/- %.2f' % ((y_pred_knn.mean()),y_pred_knn.std()))
print('Accuracy SVC:      %.2f +/- %.2f' % ((y_pred_svc.mean()),y_pred_svc.std()))
print('Accuracy Dummy:    %.2f +/- %.2f' % ((y_pred_dummy.mean()),y_pred_dummy.std()))

Accuracy SGDC:     0.36 +/- 0.22 *** 

Accuracy NCC:      0.34 +/- 0.12
Accuracy NN:       0.34 +/- 0.21
Accuracy LOGR:     0.29 +/- 0.18 

Accuracy Adaboost: 0.28 +/- 0.10
Accuracy DT:       0.28 +/- 0.19
Accuracy RFR:      0.28 +/- 0.14 

Accuracy KNN:      0.30 +/- 0.21
Accuracy SVC:      0.23 +/- 0.15
Accuracy Dummy:    0.12 +/- 0.15


## Explainability of results

In the case of the __test group__, the __best classifier__ was `SGDClassifier`, so we are going to extract the best __features__ to understand the __behaviour__ of our predictor per class. With this in mind, we are plotting the __best predictor__ (__feature__ in this case) for each class, and their respective __coefficient or weight__.

In [9]:
max_class = sgd.coef_.argmax(axis=1)

for i in range(sgd.coef_.shape[0]):
    print('Feature:', sgd.feature_names_in_[max_class[i]])
    print('Coef: %.2f' % sgd.coef_[i,max_class[i]])

Feature: Mean Pupil diameter left
Coef: 6.40
Feature: Std Gaze point X
Coef: 17.41
Feature: Std Fixation point Y
Coef: 5.18
Feature: Std Pupil diameter left
Coef: 21.66


^^^ __The outcome of this coefficient corresponds to the weight of the feature when this label is predicted.__

***TEST GROUP ONLY HAS EMPATHY SCORES FOR THE ENTIRE LABELS***

In [10]:
df['Empathy Score'].unique()

[3, 1, 2, 0]
Categories (4, int64): [0 < 1 < 2 < 3]

In [11]:
# sgd.feature_names_in_[14]

In [12]:
# max_class = sgd.coef_.argmax(axis=1)
# max_class